# Simple Coordinate Informaion

In [1]:
import numpy as np
from astropy.coordinates import SkyCoord, Angle
import astropy.units as u
from astroquery.vizier import Vizier
from astroquery.irsa import Irsa
from __future__ import print_function
import matplotlib.pyplot as plt

/var/folders/mf/jzbqy3z14sx1s6qkkkpgs4rc0000gn/T/ipykernel_38942/3807150272.py:5: DeprecationWarning: the ``irsa`` module has been moved to astroquery.ipac.irsa, please update your imports.
  from astroquery.irsa import Irsa


WASP-39 b

In [2]:
coord1 = SkyCoord.from_name("WASP-39")

In [3]:
coord1.ra.deg

217.3267299

In [4]:
coord1.dec.deg

-3.44450085

**Don't use these coordinates! Use the same 2015.5 epoch as the APT file...**

## Query for 2MASS sources nearby
Using the mirage.catalogs.create_catalaog as an example

In [5]:
def filter_bad_ra_dec(table_data):
    """Remove targets with bad RA and/or Dec values from the input table.
    Use the column masks to find which entries have bad RA or Dec values.
    These will be excluded from the Mirage catalog.
    Parameters
    ----------
    table_data : astropy.table.Table
        Input table from e.g. 2MASS query
    Returns
    -------
    position_mask : np.ndarray
        1D boolean array. True for good sources, False for bad.
    """
    ra_mask = ~table_data['ra'].data.mask
    dec_mask = ~table_data['dec'].data.mask
    position_mask = ra_mask & dec_mask
    return position_mask


In [6]:
box_width = 2048. * 0.063 * u.arcsec ## size of LW array

In [7]:
ra_dec_string = "{}  {}".format(coord1.ra.deg, coord1.dec.deg)
query_table_all = Irsa.query_region(ra_dec_string, catalog='fp_psc', spatial='Box',
                                    width=box_width)

# Exclude any entries with missing RA or Dec values
radec_mask = filter_bad_ra_dec(query_table_all)
#query_table = query_table_all[radec_mask]

## only keep a few columns
cols_to_keep = ['designation','ra','dec','k_m']
query_table = query_table_all[cols_to_keep][cols_to_keep]

# Column names of interest
magnitude_column_names = ['j_m', 'h_m', 'k_m']
#query_table, magnitude_column_names


In [8]:
len(query_table)

5

In [9]:
query_table

designation,ra,dec,k_m
,deg,deg,mag
object,float64,float64,float64
14292106-0326284,217.337769,-3.441249,14.781
14291840-0326403,217.326702,-3.444534,10.202
14291673-0327377,217.319740,-3.460491,15.423
14292130-0327190,217.338790,-3.455293,14.146
14291841-0327431,217.326719,-3.461993,15.604


### Exclude the target from the catalog

In [10]:
use_pts = query_table['designation'] != "14291840-0326403"
use_table = query_table[use_pts]
#use_table[cols_to_print]

Print to put in the Yaml file

Compact version

In [11]:
cols_to_print = ['ra','dec','k_m']
for oneCol in cols_to_print:
    outArr = np.array(use_table[oneCol])
    print("[",end='')
    for ind,item in enumerate(outArr):
        if ind == len(outArr)-1:
            print("{}".format(item),end='')
        else:
            print("{} , ".format(item),end='')
    print("]")

[217.337769 , 217.31974 , 217.33879 , 217.326719]
[-3.441249 , -3.460491 , -3.455293 , -3.461993]
[14.781 , 15.423 , 14.146 , 15.604]


Long Version

In [12]:
for oneCol in cols_to_print:
    outArr = np.array(use_table[oneCol])
    print(oneCol)
    for ind,item in enumerate(outArr):
        print("- {}".format(item))


ra
- 217.337769
- 217.31974
- 217.33879
- 217.326719
dec
- -3.441249
- -3.460491
- -3.455293
- -3.461993
k_m
- 14.781
- 15.423
- 14.146
- 15.604
